In [2]:
import matlab.engine as engine

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

mdl = 'transmission_fault_detection'

In [5]:
eng = engine.connect_matlab()
eng.set_param(mdl,'FastRestart','on',nargout=0)
out = eng.sim(mdl)
g1=eng.evalin('base','g1')
g2=eng.evalin('base','g2')
g3=eng.evalin('base','g3')

sensor = pd.DataFrame({'s1':[],
                       's2':[],
                       's3':[],
                       's4':[],
                       's5':[],
                       's6':[],
                       's7':[],
                       's8':[],
                       's9':[],
                       'MDPstate':[],
})

for mk_state in range(6):
    # Clutch fault injection
    if mk_state == 0:
        clutch = np.array([-1,-1,-1,-1,-1,-1],dtype=np.int64)
    elif mk_state == 1:
        clutch = np.array([1,-1,-1,-1,-1,-1],dtype=np.int64)
    elif mk_state == 2:
        clutch = np.array([-1, 1, 1,-1,-1,-1],dtype=np.int64)
    elif mk_state == 3:
        clutch = np.array([-1,-1,-1, 1, 1, 0],dtype=np.int64)
    elif mk_state == 4:
        clutch = np.array([0, 0,-1,-1, 1, 1],dtype=np.int64)
    elif mk_state == 5:
        clutch = np.array([0,0,0,0,0,0],dtype=np.int64)

    eng.set_param(mdl+'/[A B C D E F]','Value',np.array2string(clutch),nargout=0)
    out = eng.sim(mdl)
    # Get actual drive ratio
    idx = [{'type':'.','subs':'yout'},{'type':'{}','subs':[2]},{'type':'.','subs':'Values'},{'type':'.','subs':'Data'}]
    tmp_out = out
    for tmp_idx in idx:
        tmp_out = eng.subsref(tmp_out,tmp_idx)
    actual_dr = np.array(tmp_out)

    # Get actual drive ratio time
    idx = [{'type':'.','subs':'yout'},{'type':'{}','subs':[2]},{'type':'.','subs':'Values'},{'type':'.','subs':'Time'}]
    tmp_out = out
    for tmp_idx in idx:
        tmp_out = eng.subsref(tmp_out,tmp_idx)
    actual_time = np.array(tmp_out)

    # Get gear command, there are 6 gears
    idx = [{'type':'.','subs':'yout'},{'type':'{}','subs':[4]},{'type':'.','subs':'Values'},{'type':'.','subs':'Data'}]
    tmp_out = out
    for tmp_idx in idx:
        tmp_out = eng.subsref(tmp_out,tmp_idx)
    gear_command = np.array(tmp_out).reshape(actual_dr.shape[0],6)

    expected_dr = np.zeros(actual_dr.shape)
    for i in range(gear_command.shape[0]):
        # Gear R
        if (gear_command[i] == [1,0,0,1,0,1]).all():
            expected_dr[i,0] = -g1/(g2*(1+g1))
        # Gear 1
        elif (gear_command[i] == [0,0,1,1,0,1]).all():
            expected_dr[i,0] = g1/(g3*(1+g1))
        # Gear 2
        elif (gear_command[i] == [0,0,1,0,1,1]).all():
            expected_dr[i,0] = g1*(1+g2)/((1+g1)*(g3+g2))
        # Gear 3
        elif (gear_command[i] == [1,0,1,0,0,1]).all():
            expected_dr[i,0] = g1/(1+g1)
        # Gear 4
        elif (gear_command[i] == [0,1,1,0,0,1]).all():
            expected_dr[i,0] = (g3*(1+g1)-1)/(g3*(1+g1))
        # Gear 5
        elif (gear_command[i] == [1,1,1,0,0,0]).all():
            expected_dr[i,0] = 1.0
        # Gear 6
        elif (gear_command[i] == [1,1,0,0,0,1]).all():
            expected_dr[i,0] = (g2*(1+g1)+1)/(g2*(1+g1))
        # Gear 7
        elif (gear_command[i] == [0,1,0,0,1,1]).all():
            expected_dr[i,0] = (1+g2)/g2
        # Gear 0
        else: 
            expected_dr[i,0] = 0
    splid_len = 100
    for j in range(splid_len):
        tmp_idx = [np.where(actual_time<=4*i+4/splid_len*j)[0][-1] for i in range(9)]
        act_dr = actual_dr[tmp_idx]
        exp_dr = expected_dr[tmp_idx]
        tmp_sensor=pd.DataFrame(act_dr.T,columns=['s1','s2','s3','s4','s5','s6','s7','s8','s9'])
        tmp_sensor['MDPstate'] = mk_state
        sensor=pd.concat([sensor,tmp_sensor],ignore_index=True)


In [7]:
sensor.head()
# eng.quit()

,s1,s2,s3,s4,s5,s6,s7,s8,s9,MDPstate
0,0.000000,-0.261866,3.291945e-18,0.232221,0.412308,0.615385,0.854862,1.000000,1.163666,0.0
1,-0.261866,-0.261866,2.250886e-01,0.247950,0.412308,0.854862,0.854862,1.000000,1.163666,0.0
2,-0.261866,-0.261866,2.322206e-01,0.412308,0.517007,0.854862,0.870187,1.008256,1.163666,0.0
3,-0.261866,-0.261866,2.322206e-01,0.412308,0.615385,0.854862,0.968973,1.100110,1.166024,0.0
4,-0.261866,-0.238882,2.322206e-01,0.412308,0.615385,0.854862,1.000000,1.163666,1.223792,0.0


In [6]:
tmp_sensor = pd.read_csv('sensor.csv',index_col=0)
# tmp_sensor.head()
tmp_col = tmp_sensor.columns[0:9]
tmp_slice = tmp_sensor.loc[tmp_sensor['MDPstate']==1]

In [13]:
tmp_sensor

,s1,s2,s3,s4,s5,s6,s7,s8,s9,MDPstate
0,0.000000,-0.261866,3.291945e-18,0.232221,0.412308,0.615385,0.854862,1.000000,1.163666,0.0
1,-0.261866,-0.261866,2.250886e-01,0.247950,0.412308,0.854862,0.854862,1.000000,1.163666,0.0
2,-0.261866,-0.261866,2.322206e-01,0.412308,0.517007,0.854862,0.870187,1.008256,1.163666,0.0
3,-0.261866,-0.261866,2.322206e-01,0.412308,0.615385,0.854862,0.968973,1.100110,1.166024,0.0
4,-0.261866,-0.238882,2.322206e-01,0.412308,0.615385,0.854862,1.000000,1.163666,1.223792,0.0
...,...,...,...,...,...,...,...,...,...,...
595,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0
596,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0
597,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0
598,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.0


In [12]:
tmp_slice.head()

,s1,s2,s3,s4,s5,s6,s7,s8,s9,MDPstate


In [10]:
tmp_slice.drop(tmp_slice.index,inplace=True)

/tmp/ipykernel_135646/1174696646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_slice.drop(tmp_slice.index,inplace=True)


In [4]:
train_data = sensor[sensor.columns[0:9]].values
train_label = sensor['MDPstate'].values
train_label = keras.utils.to_categorical(train_label)
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)

In [10]:
sensor.drop(sensor.index,inplace=True)

In [5]:
# x_train,x_test,y_train,y_test = train_test_split(train_data, train_label, test_size=0.33)
x_train = x_train.reshape((402,1,9))
x_test = x_test.reshape((198,1,9))

In [19]:
# train_data.shape
y_train = y_train.reshape((402,1,6))

In [21]:
model = keras.Sequential(
    [
#        layers.LSTM(32,return_sequences=True,activation='relu',input_shape=[1,9]),
#        layers.LSTM(32),
        layers.Dense(32,activation='relu',input_shape=[1,9]),
        layers.Dense(32),
        layers.Dropout(0.2),
        layers.Dense(6,activation='sigmoid')
    ]
)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1, 32)             320       
                                                                 
 dense_7 (Dense)             (None, 1, 32)             1056      
                                                                 
 dropout_2 (Dropout)         (None, 1, 32)             0         
                                                                 
 dense_8 (Dense)             (None, 1, 6)              198       
                                                                 
Total params: 1,574
Trainable params: 1,574
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = model.fit(x_train,y_train,
                    epochs=100,
                    validation_split=0.1)

Epoch 1/100
12/12 [==============================] - 1s 18ms/step - loss: 1.7616 - accuracy: 0.1690 - val_loss: 1.6274 - val_accuracy: 0.1463
Epoch 2/100
12/12 [==============================] - 0s 3ms/step - loss: 1.6076 - accuracy: 0.3269 - val_loss: 1.5013 - val_accuracy: 0.7561
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 1.5008 - accuracy: 0.4931 - val_loss: 1.3835 - val_accuracy: 0.7561
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 1.3815 - accuracy: 0.6620 - val_loss: 1.2493 - val_accuracy: 0.7561
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: 1.2630 - accuracy: 0.7313 - val_loss: 1.1134 - val_accuracy: 0.9268
Epoch 6/100
12/12 [==============================] - 0s 3ms/step - loss: 1.1439 - accuracy: 0.8199 - val_loss: 0.9741 - val_accuracy: 0.9268
Epoch 7/100
12/12 [==============================] - 0s 3ms/step - loss: 1.0141 - accuracy: 0.8809 - val_loss: 0.8539 - val_accuracy: 1.0000
Epoch 8/100


In [42]:
label_pro = model.predict(x_test)
label_pre = label_pro.argmax(axis=-1)
label_test = y_test.argmax(axis=-1)
acc = np.sum(label_pre==label_test) / label_pre.shape[0]
print("Accuracy is:", acc)


7/7 [==============================] - 0s 2ms/step
Accuracy is: 0.9848484848484849


In [56]:
from gym import spaces

In [57]:
space = spaces.Tuple([spaces.Discrete(2) for _ in range(12)])

In [58]:
tmp_s = space.sample()
print(tmp_s)

(0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1)


In [63]:
for i in np.where(np.array(tmp_s))[0]:
    print(i)

2
5
6
7
8
10
11


In [7]:
import numpy as np
import pandas as pd
import traci
traci.start(["sumo-gui", "-c", "map/3km_1week/osm.sumocfg","--threads","6"])

 Retrying in 1 seconds


(20, 'SUMO 1.15.0')

In [188]:
traci.close()
# traci.vehicle.getMaxSpeed(typeID='lorry')

In [84]:
traci.vehicle.add(vehID='lorry',routeID='Factory0_to_Factory0', typeID='lorry')
traci.vehicle.setParkingAreaStop(vehID='lorry',stopID='Factory0')

In [182]:
traci.vehicle.changeTarget(vehID='lorry', edgeID='Factory0')
traci.vehicle.setParkingAreaStop(vehID='lorry',stopID='Factory3',duration=0)
traci.vehicle.setParkingAreaStop(vehID='lorry',stopID='Factory0')

In [156]:
edgeID = traci.vehicle.getRoadID(vehID='lorry')
traci.vehicle.setSpeed(vehID='lorry',speed=0)


In [152]:
traci.vehicle.getLanePosition(vehID='lorry')

497.02481177524834

In [157]:
traci.vehicle.setStop(vehID='lorry',edgeID=edgeID,pos=traci.vehicle.getLanePosition(vehID='lorry')+10)

In [163]:
traci.vehicle.resume(vehID='lorry')

In [88]:
for i in range(10):
    traci.simulationStep()
    time_step = traci.simulation.getTime()

In [187]:
for _ in range(100):
    traci.simulationStep()
    speed = traci.vehicle.getSpeed(vehID='lorry')
    if speed > 19:
        print(speed)